In [166]:
from bs4 import BeautifulSoup
import urllib3
from urllib.request import urlopen
import regex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#this is a basic scrapper which uses BS3 and BS4 to scrape and filter data

def line_scrape (lookup, data, skip): #take a lookup tag, datafile, and no of lines to skip
    arr1 = np.array([])
    counter = 0
    for line in data.splitlines():
        if lookup not in line: #check if tag present, if not, skip line
            counter +=1
        else:
            counter = counter + skip #if yes, skip n lines
            entry = ((data.splitlines()[counter]))
            arr1 = np.append(arr1,entry) #aggregate data
            arr1 = arr1.reshape((-1, 1)) #turn into column
    return arr1
            
            

def scrape_row (link): 
    link = urlopen(link)#take a link of all htmls
    soup = BeautifulSoup(link, "lxml") #set decoding system
    entry = np.array([]) #create empty array for storing data
    arr_final = np.array([])
    print (link)
    datasize = 0 #tell me how many links you succesfully got the data for
    for link in soup.findAll('a', attrs={'href': regex.compile("^https://www.bgf.co.uk/our-portfolio/")}): #for every link in list of all htmls
        link_final = (link.get('href'))
        new_page=urlopen(link_final)
        new_soup = BeautifulSoup(new_page, "lxml") #decode the page itself
        data1 = new_soup.find("body").get_text() #isolate text
        entry1 = line_scrape ('Shape', data1, 9)
        entry1 = entry1.astype(str)
        if entry1.size ==0:
            print (datasize)
            entry1 = 'N/A'
            
        entry2 = line_scrape ('Total BGF investment', data1, 1)
        if len (entry2)==(0):
            newline =  np.array([])
            entry2 = line_scrape ('£', data1, 0)
            line = np.array2string(entry2)
            iden = line_scrape ('Shape', data1, 9)
            position = line.find('£')
            if position != -1:
                inventive_arr = np.array([])
                for i in range (4):
                    position_new = (position + i)
                    z = line[position_new]
                    inventive_arr = np.append(inventive_arr, z)
                entry2 = inventive_arr.astype(str)
                entry2=' '.join(entry2)
            else:
                entry2 = 'N/A'     
            
        entry3 = line_scrape ('Invested since', data1, 1)
        if len (entry3)==(0):
            entry3 = line_scrape ('Investment between', data1, 1)
            if len (entry3)==(0):
                entry3 = line_scrape ('Investment', data1, 1) 
                if len (entry3)==(0):
                    entry3 = line_scrape ('Invested', data1, 1) 
                    if len (entry3)==0:
                        print ('no investment')
                        iden = line_scrape ('Shape', data1, 9)
                        entry3 = 'N/A'
                        print (iden)
                    
                    
        entry = np.append(entry,entry1)
        entry = np.append(entry,entry2)
        entry = np.append(entry,entry3)
        entry = np.array2string(entry) #concatenate all entries and unify type of array into string
        entry = (entry + '|') #append with delimiter
        
        arr_final= np.append(arr_final,entry) #make a string array
        entry = np.array([])
        datasize = (datasize+1)
    return arr_final

 
x = scrape_row ("https://www.bgf.co.uk/full-portfolio-archive/")
print (x)


["['3sun Group' '£10m' 'Feb 2014']|" "['Abacus' '£4.15m' 'Aug 2013']|"
 "['Accsys Group' '£12m' 'Mar 2017']|"
 "['Acro' '\\xa0£7.8m' 'Nov 2015 – Sep 2017']|"
 "['ACS Clothing' '£25.3m' 'Jan 2014']|"
 "['AdEPT Telecom' '£9m' 'Sep 2016']|" "['ADEPT4 plc' '£5m' 'Jun 2016']|"
 "['Adestra' '£5.7m\\xa0' 'Jan 2016']|"
 "['AimBrain' '£ 4 m  ' 'Jun 2017']|"
 "['Amazon Filters' 'N/A' 'September 2018']|"
 "['Anstey Horne' '£6.6m' 'Mar 2016']|"
 "['Apex Housing Group' '£6.25m' 'June 2018']|"
 "['APS Group' 'N/A' 'Dec 2017']|"
 "['APSU' '£7m' 'Oct 2014 – Nov 2016' '']|"
 "['Arcinova' '£5m' 'June 2018']|" "['Aubin' '£2.3m\\xa0' 'Feb 2013']|"
 "['AuditComply' 'N/A' 'Dec 2017']|" "['Bar Soba' '£3m' 'Nov 2016']|"
 "['Barburrito' '£12.6 million' 'Mar 2012']|"
 "['Benefex' '£5.5 million' 'Oct 2011']|"
 "['BHR Group' '£3.1m' 'Jun 2014']|"
 "['BigBlu Broadband plc' '£14.5m' 'Apr 2016']|" "['' 'N/A' 'Jan 2019']|"
 "['Bob & Berts' '£2m' 'Aug 2017']|" "['Braidwater' 'N/A' 'Nov 2015']|"
 "['Brindley Healthcare

In [174]:
np.savetxt(fname='meow.txt',fmt='%s', X=x, delimiter='|')